# Init 

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import glob
from os import path
from IPython.display import Video

from utils import *
from pre_processing import *
from change_detection import *
from object_classification import *
from post_processing import *

In [333]:
def get_frames(file_name ,nf1):
    cap = cv2.VideoCapture(file_name)
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret:
            frame_num = cap.get(cv2.CAP_PROP_POS_FRAMES)
            gray = cv2.cvtColor(frame, cv2.IMREAD_COLOR)
            if int(frame_num) == nf1:
                f1 = gray
                return f1
            cv2.waitKey(30)
        else:
            return f1

In [353]:
pre_image = get_frames('data/videos/approved1.mp4', 162)

In [354]:
cv2.imwrite('data/preprocessing_images/image6.jpg', pre_image)

True

# Load Video 

In [2]:
video_name = 'approved1'
video_path = path.join('data', 'videos', f'{video_name}.mp4')
video_det_path = path.join('data', 'detected_videos', f'{video_name}.mp4')

In [3]:
vid = load_video(video_path=video_path)
Video(video_path, height=400, width=600)

# Preprocessing

In [444]:
import object_classification, utils
import importlib
importlib.reload(object_classification)
importlib.reload(utils)

<module 'utils' from 'C:\\Users\\alon2\\Desktop\\CV_Final\\utils.py'>

> Histograms Calculation
--

#### Preprocessed images outline

In [441]:
images_path = "data/object_outline/"
image_list = utils.load_images(images_path)

#### Calculating the outline histograms

In [357]:
window_size = 15
histograms_mean_dict = {"RED":[], "GREEN":[], "BLUE":[]}
for img_num, img in enumerate(image_list):
    r_histograms_mean_list = object_classification.generate_patches_histograms_mean(img[:,:,0], window_size)
    g_histograms_mean_list = object_classification.generate_patches_histograms_mean(img[:,:,1], window_size)
    b_histograms_mean_list = object_classification.generate_patches_histograms_mean(img[:,:,2], window_size)
    histograms_mean_dict["RED"].append(r_histograms_mean_list)
    histograms_mean_dict["GREEN"].append(g_histograms_mean_list)
    histograms_mean_dict["BLUE"].append(b_histograms_mean_list)

#### Clustering the hitograms mean and calculating the mean for each cluster

In [370]:
bins = 2
allowed_histogram_means = object_classification.calculate_allowed_histograms_means(histograms_mean_dict, bins)

In [403]:
allowed_histogram_means

{'RED': [145.97060582144468, 171.66362499999997],
 'GREEN': [146.68428421481053, 173.34629166666656],
 'BLUE': [139.02536768604205, 166.758625]}

> Object Size calculation
--

#### Creating a grid of the input image

In [372]:
images_path = "data/preprocessing_images/"
image_to_grid = utils.load_images(images_path)[0] # taking the first image based on the knowledge all images sizes are the same

In [373]:
grid_size = 5 # grid the image with boxes of 20X20
image_x_grid = np.arange(0, image_to_grid.shape[1], grid_size)
image_y_grid = np.arange(0, image_to_grid.shape[0], grid_size)

#### Creating a database for approved object size information

In [404]:
first_object_location = [100, 325, 260, 360]
object_size_info = object_classification.get_grids(image_x_grid, image_y_grid, first_object_location)

In [405]:
object_size_info

{'middle_grid': [42.5, 62.0],
 'grid_size': 836,
 'grid_coordinates': {'X': (array([21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37,
          38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54,
          55, 56, 57, 58, 59, 60, 61, 62, 63, 64], dtype=int64),),
  'Y': (array([53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69,
          70, 71], dtype=int64),)}}

In [410]:
first_approved_object_info = {"RED": allowed_histogram_means["RED"],
                             "GREEN": allowed_histogram_means["GREEN"],
                             "BLUE": allowed_histogram_means["BLUE"],
                             "location": [object_size_info]}

In [411]:
first_approved_object_info

{'RED': [145.97060582144468, 171.66362499999997],
 'GREEN': [146.68428421481053, 173.34629166666656],
 'BLUE': [139.02536768604205, 166.758625],
 'location': [{'middle_grid': [42.5, 62.0],
   'grid_size': 836,
   'grid_coordinates': {'X': (array([21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37,
            38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54,
            55, 56, 57, 58, 59, 60, 61, 62, 63, 64], dtype=int64),),
    'Y': (array([53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69,
            70, 71], dtype=int64),)}}]}

# Change Detection 

In [376]:
det_vid = change_dection(vid)

In [377]:
play_video_by_images(det_vid, frame_rate=40)

AttributeError: 'NoneType' object has no attribute 'shape'

In [378]:
save_video(det_vid, video_det_path)
# Video(video_det_path, height=400, width=600)

AttributeError: 'NoneType' object has no attribute 'shape'

# Object Classification 

In [379]:
# import image got from change detection phase
detected_images = "data/object_classification/"
loaded_detected_images = utils.load_images(detected_images)

In [448]:
detected_objects = [{'object1': [520, 660, 260, 320], 'object2': [860, 1040, 470, 590],
                    'object3':[600, 800, 0, 130], 'object4': [1040, 1120, 130, 270]}]

In [449]:
for idx, img in enumerate(loaded_detected_images):
    detected_objects_location = detected_objects[idx]
    num_of_objects = len(detected_objects_location)
    objects_info = object_classification.get_detected_objects_info(img, detected_objects_location, image_x_grid, image_y_grid, window_size)
    approved_objects = object_classification.match_with_approved_objects(num_of_objects, objects_info, first_approved_object_info)

In [451]:
for i in approved_objects:
    if f"object{i+1}" in detected_objects[0].keys():
        print(f"object{i+1} is approved")

object2 is approved


In [ ]:
#TODO
# save the image with red squares wrapping the unapproved objects and blue squares wrapping approved objects

# Postprocessing 